In [562]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import networkx as nx

In [563]:
# [ 16,  52,  63,  66,  90,  99, 123, 143, 179, 191, 196]

In [564]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60
ins = 10
task = pd.read_csv(f"../../data/utilization/utilization_40_{ins}_task.csv")
network = pd.read_csv(f"../../data/utilization/utilization_40_{ins}_topo.csv")

for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

In [565]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [566]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(eval(row['link'])[0], {})
    net_var[eval(row['link'])[0]]['dproc'] = np.ceil(row['t_proc'])
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

## Create mapping from Link to index
link_to_index = {}
index_to_link = {}

counter = 0
for _, row in network.iterrows():
    link = row['link']
    link_to_index[link] = counter
    index_to_link[counter] = link
    counter += 1

In [567]:
task_attr = {}
task_var = {}
graph = nx.from_numpy_array(net)
## Assume task is strictly periodic
for i, row in task.iterrows():
    task.loc[i,'route'] = str(nx.shortest_path(graph, row['src'], eval(row['dst'])[0]))
    task_var.setdefault(i, {})
    route = eval(task.loc[i, 'route'])
    task_attr.setdefault(i, {})
    task_attr[i]['route'] = route
    task_attr[i]['p'] = row['period']
    task_attr[i]['d'] = row['deadline']
    for _i, a in enumerate(route[:-1]):
        link = str((a, route[_i + 1]))
        task_var[i].setdefault(link, {})
        task_var[i][link]['dtrans'] = np.ceil(row['size'] * 8)
        if _i == 0:
            ## This one must not cantains processing delay
            task_var[i][link]['D'] = task_var[i][link]['dtrans']
        else:
            task_var[i][link]['D'] = task_var[i][str((route[_i - 1], a))]['D'] \
            + net_var[eval(link)[0]]['dproc'] + task_var[i][link]['dtrans']


In [568]:
def mod(dividend, divisor):
    result = dividend % divisor
    return result

In [569]:
def collision_free_interval(i, j, link):
    global config_hash
    offset_j = config_hash[j]
    tau = offset_j + (task_var[j][link]['D'] - task_var[j][link]['dtrans'])\
        - (task_var[i][link]['D'] - task_var[i][link]['dtrans'])

    phi_interval = (
        tau - task_var[i][link]['dtrans'],
        tau + task_var[j][link]['dtrans']
    )
    return phi_interval

In [570]:
scheduled_set = set()
config_hash = dict()

In [571]:
a = 100

if a > 0:
    print('a')
elif a > 2:
    print('b')
else:
    print('c')

a


In [572]:
## Main algorithm
for epi in tqdm(task_attr):
    divider_interval_set = []
    ## Collect all the collision intervals
    for i in scheduled_set:
        for link in [link_i for link_i in task_var[i] for link_epi in task_var[epi] if link_i == link_epi]:
            divider = np.gcd(task_attr[i]['p'], task_attr[epi]['p'])
            interval = collision_free_interval(epi, i, link)
            interval_regulated = (mod(interval[0], divider), mod(interval[1], divider))
            if interval_regulated[0] >= interval_regulated[1]:
                divider_interval_set.append((divider, (-1, interval_regulated[1]), (i,link)))
                divider_interval_set.append((divider, (interval_regulated[0], divider), (i,link)))                
            else:
                divider_interval_set.append((divider, interval_regulated, (i,link)))
    end_link = str((task_attr[epi]['route'][-2] ,task_attr[epi]['route'][-1]))
    offset_upper_bound = task_attr[epi]['p'] - task_var[epi][end_link]['D']
    injection_time = 0
    trial_counter = 0
    divider_interval_set = sorted(divider_interval_set, key=lambda x:x[1][1], reverse=False)
    cyclic_index = 0
    
    while trial_counter < len(divider_interval_set):
        divider, interval, _ = divider_interval_set[cyclic_index]
        assert interval[0] < interval[1], divider_interval_set
        regulated_offset = mod(injection_time, divider)
        if regulated_offset > interval[0] and regulated_offset < interval[1]:
            if interval[1] > injection_time:
                injection_time = interval[1]
            else:
                injection_time += interval[1] - interval[0]
#             print(injection_time)
            trial_counter = 1
            if injection_time > offset_upper_bound:
                raise Exception(f"Flow {epi} cannot be scheduled")
                break
        else:
            trial_counter += 1
        cyclic_index += 1
        if cyclic_index == len(divider_interval_set):
            cyclic_index = 0
    scheduled_set.add(epi)
    config_hash[epi] = injection_time
#     print(f"Flow {epi} injection time {injection_time}")

 37%|████████████████████▊                                   | 73/197 [00:00<00:00, 7471.73it/s]


Exception: Flow 73 cannot be scheduled

In [ ]:
## Check if the schedule is collision free
GCL = {}
for epi in task_var:
    for link in task_var[epi]:
        GCL.setdefault(link, [])
        GCL[link].append((config_hash[epi] + task_var[epi][link]['D'] - task_var[epi][link]['dtrans'], 
                              config_hash[epi] + task_var[epi][link]['D'], 0, epi))
for link in GCL:
    GCL[link] = sorted(GCL[link], key=lambda x: x[0], reverse=False)
    temp = GCL[link]
    for i, row in enumerate(temp[:-1]):
        assert row[1] <= temp[i + 1][0], f"Overlap {link}\n Entry1 {row}\n Entry2 {temp[i + 1]}\n"